##Installation

In [ ]:
!pip install lightfm
!pip install gdown
!pip install recommenders
!pip install rankfm
!pip install -U scikit-learn
!pip install tf_slim
!pip install numpy==1.22
!pip install LibRecommender
!pip install implicit


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.1/310.1 KB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.16-cp38-cp38-linux_x86_64.whl size=917979 sha256=2ff6973d37cd4cb23819635263ccb8b082c3bf2b8df43fcbb3c27753a39944f0
  Stored in directory: /root/.cache/pip/wheels/ec/bb/51/9c487d021c1373b691d13cadca0b65b6852627b1f3f43550fa
Successfully built lightfm
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.0/339.0 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.2/355.2 KB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
os.kill(os.getpid(), 9)

## Data Loading

In [ ]:

import pandas as pd
import os
from os.path import exists
import zipfile
import numpy as np
from datetime import datetime

In [ ]:
SEED=42

In [ ]:
# !gdown https://drive.google.com/uc?id=1kl65YOvoSAMSgszQQbua2q4Zwe1HgPct
# !unzip -o "data.zip"  -d  "/content"

# movie_10k_df = pd.read_csv("movies_min.csv")
# amazon_df =pd.read_csv('amazon_min.csv')
# book_df=pd.read_csv("book_min.csv")
# food_df = pd.read_csv('food_min.csv')
# ecommerce_df=pd.read_csv('ecom_min.csv')
# ciao_df=pd.read_csv('ciao_min.csv')

In [ ]:
TIMESTAMP=None

In [ ]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!gdown https://drive.google.com/uc?id=1kl65YOvoSAMSgszQQbua2q4Zwe1HgPct

Downloading...
From: https://drive.google.com/uc?id=1kl65YOvoSAMSgszQQbua2q4Zwe1HgPct
To: /content/data.zip
100% 4.16M/4.16M [00:00<00:00, 282MB/s]


In [ ]:
!unzip -o "data.zip"  -d  "/content"

Archive:  data.zip
  inflating: /content/amazon_min.csv  
  inflating: /content/book_min.csv   
  inflating: /content/ciao_min.csv   
  inflating: /content/ecom_min.csv   
  inflating: /content/food_min.csv   
  inflating: /content/movies_min.csv  


#### Name of the file where user item interaction data avaiable?
- Column name should be in following name and order
- [ **itemID, userID , rating , timestamp** ]
- No issue if there is no column for **timestamp**

In [ ]:
filename="movies_min.csv" #enter your filename with file path

##### Run below cell

In [ ]:
rating_df = pd.read_csv(filename)


In [ ]:
NUM_OF_COLS=rating_df.shape[1]


In [ ]:
import time 
if  'timestamp' in rating_df.columns:
  rating_df =rating_df[[ 'userID', 'itemID', 'rating','timestamp']]
  rating_df.columns =[ 'userID', 'itemID', 'rating','timestamp']
else:

  obj = time.gmtime(0)
  epoch = time.asctime(obj)
  curr_time = round(time.time()*1000)
  rating_df['timestamp']=curr_time
  rating_df =rating_df[[ 'userID', 'itemID', 'rating','timestamp']]
  rating_df.columns =[ 'userID', 'itemID', 'rating','timestamp']

In [ ]:
u_users=list(rating_df.userID.unique())

In [ ]:
u_items=list(rating_df.itemID.unique())

### Setup

In [ ]:
# Evaluation metrics
import libreco
import tensorflow as tf
import torch
from libreco.data import DatasetFeat, DataInfo
from libreco.data import  split_by_ratio
from libreco.data import split_by_ratio_chrono, DatasetPure, DatasetFeat
from libreco.evaluation import evaluate
from libreco.prediction import predict_data_with_feats
from libreco.algorithms import (
    ALS
)
import time
import pandas as pd

metrics = [
    "precision",
    "recall",
    "map",
    "ndcg"
]

# Call Librecommender API
# datasettype ="pure" or "feat"
def run(rating_df,datasettype="pure", algo=["als"],hyperparam={"als":{ "embed_size":16,
        "n_epochs":10,
        "reg":5.0,
        "alpha":10,
        "use_cg":False,
        "n_threads":1,
        "seed":42}}, neg_samples=True):
    train_data, test_data, data_info=None,None,None
    tf.compat.v1.reset_default_graph()
    if(datasettype=="pure"):
      train_data, test_data, data_info = preprocessingPure(rating_df, neg_samples)
    else :
      train_data, test_data, data_info = preprocessingFeature(rating_df, neg_samples) 
    models = dict()
    evals = dict()
    for i in algo:
        if i == 'Caser':
            
            als, eval, algo_data_info = Caser(train_data, test_data, data_info,hyperparam["Caser"])
            models['Caser'] = als
            evals['Caser'] = eval
            print("Evaluation Results for Caser:")
            print(evals['Caser'])
        elif i == 'rnn4rec':
            rnn, eval, algo_data_info = rnn4rec(train_data, test_data, data_info,hyperparam["rnn4rec"])
            models['rnn4rec'] = rnn
            evals['rnn4rec'] = eval
            print("Evaluation Results for RNN4REC:")
            print(evals['rnn4rec'])
        
        else:
            graphsage, eval, algo_data_info = graphSage(train_data, test_data, data_info,hyperparam["graphSage"])
            models['graphSage'] = graphsage
            evals['graphSage'] = eval
            print("Evaluation Results for GraphSage:")
            print(evals['graphSage'])
          
    return models, evals


# data preprocessing
def preprocessingPure(rating_df, neg_samples=True):
    data = rating_df.copy()
    data.columns = ['user', 'item', 'label', 'timestamp']
    col_user = ["user"]
    col_item = ["item"]
    col_dense = ["label"]
    train, test = split_by_ratio(data, test_size=0.2)
    train_data, data_info = DatasetPure.build_trainset(
        train,
        shuffle=False,
    )
    test_data = DatasetPure.build_testset(test, shuffle=False)
    if neg_samples:
        train_data.build_negative_samples(
            data_info, item_gen_mode="random", num_neg=1, seed=2020
        )
        test_data.build_negative_samples(
            data_info, item_gen_mode="random", num_neg=1, seed=2222
        )
    return train_data, test_data, data_info
# data preprocessing
def preprocessingFeature(rating_df, neg_samples=True):
    data = rating_df.copy()
    data.columns = ['user', 'item', 'label', 'timestamp']
    col_user = ["user"]
    col_item = ["item"]
    col_dense = ["label"]
    train, test = split_by_ratio(data, test_size=0.2)
    train_data, data_info = DatasetFeat.build_trainset(
        train,
        shuffle=False,
    )
    test_data = DatasetFeat.build_testset(test, shuffle=False)
    if neg_samples:
        train_data.build_negative_samples(
            data_info, item_gen_mode="random", num_neg=1, seed=2020
        )
        test_data.build_negative_samples(
            data_info, item_gen_mode="random", num_neg=1, seed=2222
        )
    return train_data, test_data, data_info


# Recommend items to user
def recommend(model, userid, k=10):
    return model.recommend_user(user=userid, n_rec=k)

# Find similar items
def similarItems(model, itemids, k=10):
    return model.search_knn_items(item=itemids, k=k)


# Model : Alternating Least Square (ALS)
def alteringleastsquare(train_data, test_data, data_info,hyperparam={"embed_size":16,
        "n_epochs":10,
        "reg":5.0,
        "alpha":10,
        "use_cg":False,
        "n_threads":1,
        "seed":42}):
    als = libreco.algorithms.ALS(
        "ranking",
        data_info,
        embed_size=hyperparam['embed_size'],
        n_epochs=hyperparam['n_epochs'],
        reg=hyperparam['reg'],
        alpha=hyperparam['alpha'],
        use_cg=hyperparam['use_cg'],
        n_threads=hyperparam['n_threads'],
        seed=hyperparam['seed'],
    )
    als.fit(
        train_data,
        verbose=2,
        eval_data=test_data,
        metrics=metrics,
    )
    result = evaluate(model=als, data=test_data, metrics=metrics)
    return als, result, data_info

def graphSage(train_data, test_data, data_info,hyperparam={ "loss_type":"max_margin",
        "paradigm":"i2i",
        "embed_size":16,
        "n_epochs":2,
        "lr":3e-4,
        "lr_decay":False,
        "reg":None,
        "batch_size":2048,
        "num_neg":1,
        "dropout_rate":0.0,
        "num_layers":1,
        "num_neighbors":10,
        "num_walks":10,
        "sample_walk_len":5,
        "margin":1.0,
        "sampler":"random",
        "start_node":"random",
        "focus_start":False,
        "seed":42}  ):
    graphsage = libreco.algorithms.GraphSage(
        "ranking",
        data_info,
        loss_type=hyperparam["loss_type"],
        paradigm=hyperparam["paradigm"],
        embed_size=hyperparam["embed_size"],
        n_epochs=hyperparam["n_epochs"],
        lr=hyperparam["lr"],
        lr_decay=hyperparam["lr_decay"],
        reg=hyperparam["reg"],
        batch_size=hyperparam["batch_size"],
        num_neg=hyperparam["num_neg"],
        dropout_rate=hyperparam["dropout_rate"],
        num_layers=hyperparam["num_layers"],
        num_neighbors=hyperparam["num_neighbors"],
        num_walks=hyperparam["num_walks"],
        sample_walk_len=hyperparam["sample_walk_len"],
        margin=hyperparam["margin"],
        sampler=hyperparam["sampler"],
        start_node=hyperparam["start_node"],
        focus_start=hyperparam["focus_start"],
        seed=hyperparam["seed"],
    )
    graphsage.fit(
        train_data,
        verbose=2,
        shuffle=True,
        eval_data=test_data,
        metrics=metrics,
    )
    result = evaluate(model=graphsage, data=test_data, metrics=metrics)
    return graphsage, result, data_info

def bayesianpersonalizedranking(train_data, test_data, data_info,hyperparam={ "loss_type":"bpr",
        "paradigm":"i2i",
        "embed_size":16,
        "n_epochs":2,
        "lr":3e-4,
        "lr_decay":False,
        "reg":None,
        "batch_size":2048,
        "num_neg":1,
        "use_tf":True}  ):
    bpr = libreco.algorithms.BPR(
        "ranking",
        data_info,
        loss_type=hyperparam["loss_type"],
        paradigm=hyperparam["paradigm"],
        embed_size=hyperparam["embed_size"],
        n_epochs=hyperparam["n_epochs"],
        lr=hyperparam["lr"],
        lr_decay=hyperparam["lr_decay"],
        reg=hyperparam["reg"],
        batch_size=hyperparam["batch_size"],
        num_neg=hyperparam["num_neg"],
        use_tf=hyperparam["use_tf"],
    )
    bpr.fit(
        train_data,
        verbose=2,
        num_threads=4,
        eval_data=test_data,
        metrics=metrics,
        optimizer="adam",
    )
    result = evaluate(model=bpr, data=test_data, metrics=metrics)
    return bpr, result, data_info

def rnn4rec(train_data, test_data, data_info,hyperparam={ "rnn_type":"gru",
        "loss_type":"cross_entropy",
        "embed_size":16,
        "n_epochs":2,
        "lr":3e-4,
        "lr_decay":False,
        "reg":None,
        "batch_size":2048,
        "num_neg":1,
        "use_tf":True,
        "dropout_rate":0.0,
        "recent_num":10,
        "tf_sess_config":None}  ):
    rnn = libreco.algorithms.RNN4Rec(
        "ranking",
        data_info,
        loss_type=hyperparam["loss_type"],
        rnn_type=hyperparam["rnn_type"],
        embed_size=hyperparam["embed_size"],
        n_epochs=hyperparam["n_epochs"],
        lr=hyperparam["lr"],
        lr_decay=hyperparam["lr_decay"],
        reg=hyperparam["reg"],
        batch_size=hyperparam["batch_size"],
        num_neg=hyperparam["num_neg"],
        recent_num=hyperparam["recent_num"],
        tf_sess_config=hyperparam["tf_sess_config"],
    )
    rnn.fit(
        train_data,
        verbose=2,
        shuffle=True,
        eval_data=test_data,
        metrics=metrics,
    )
    result = evaluate(model=rnn, data=test_data, metrics=metrics)
    return rnn, result, data_info

def Caser(train_data, test_data, data_info,hyperparam={ 
        "loss_type":"cross_entropy",
        "embed_size":16,
        "n_epochs":2,
        "lr":3e-4,
        "lr_decay":False,
        "reg":None,
        "batch_size":2048,
        "num_neg":1,
        "use_bn":True,  
        "nh_filters":16,
        "nv_filters":4,
        "dropout_rate":0.0,
        "recent_num":10,
        "tf_sess_config":None}  ):
    Caser = libreco.algorithms.Caser(
        "ranking",
        data_info,
        loss_type=hyperparam["loss_type"],

        embed_size=hyperparam["embed_size"],
        n_epochs=hyperparam["n_epochs"],
        lr=hyperparam["lr"],
        lr_decay=hyperparam["lr_decay"],
        reg=hyperparam["reg"],
        batch_size=hyperparam["batch_size"],
        nh_filters=hyperparam["nh_filters"],
        nv_filters=hyperparam["nv_filters"],
        num_neg=hyperparam["num_neg"],
        recent_num=hyperparam["recent_num"],
        tf_sess_config=hyperparam["tf_sess_config"],
    )
    Caser.fit(
        train_data,
        verbose=2,
        shuffle=True,
        eval_data=test_data,
        metrics=metrics,
    )
    result = evaluate(model=Caser, data=test_data, metrics=metrics)
    return Caser, result, data_info





    


##  RNN4REC

In [ ]:
IS_SEQUENTIAL="YES"

In [ ]:

models, evals = run(rating_df, algo=["rnn4rec"], hyperparam={"rnn4rec":{ "rnn_type":"gru",
        "loss_type":"cross_entropy",
        "embed_size":16,
        "n_epochs":2,
        "lr":3e-4,
        "lr_decay":False,
        "reg":None,
        "batch_size":2048,
        "num_neg":1,
        "use_tf":True,
        "dropout_rate":0.0,
        "recent_num":10,
        "tf_sess_config":None}},neg_samples=False)


total params: 35,928 | embedding params: 34,136 | network params: 1,792
Training start time: 2023-02-07 19:06:04


train: 100%|██████████| 38/38 [00:03<00:00, 11.57it/s]


Epoch 1 elapsed: 3.308s
	 train_loss: 0.6779


eval_listwise: 100%|██████████| 31/31 [00:00<00:00, 208.76it/s]


	 eval precision@10: 0.0834
	 eval recall@10: 0.0467
	 eval map@10: 0.1874
	 eval ndcg@10: 0.2710


...

Epoch 2 elapsed: 2.770s
	 train_loss: 0.6273


eval_listwise: 100%|██████████| 31/31 [00:00<00:00, 108.96it/s]


	 eval precision@10: 0.1150
	 eval recall@10: 0.0665
	 eval map@10: 0.2158
	 eval ndcg@10: 0.3239


eval_listwise: 100%|██████████| 135/135 [00:00<00:00, 506.61it/s]


Evaluation Results for RNN4REC:
{'precision': 0.11495227995758218, 'recall': 0.0665062245327051, 'map': 0.21576215304078505, 'ndcg': 0.32392041512799913}


#### Recommendations to users

In [ ]:
rnn4recmodel=models['rnn4rec']

In [ ]:
userids=u_users[0]
itemids=u_items[0]

In [ ]:
print('recommendation user '+str(userids))
results= recommend(rnn4recmodel,1, 10)
print(results)

recommendation user 196
{1: array([195, 300, 286, 462, 204, 475, 433, 657, 508, 310])}


In [ ]:
print(" 10 most similar items for item : ", similarItems(rnn4recmodel,itemids, k=10))

 3 most similar items for item 2:  [289, 568, 530, 185, 208, 141, 429, 294, 47, 202]


##  RNN4REC

In [ ]:
IS_SEQUENTIAL="YES"

In [ ]:

models, evals = run(rating_df, algo=["rnn4rec"], hyperparam={"rnn4rec":{ "rnn_type":"gru",
        "loss_type":"cross_entropy",
        "embed_size":16,
        "n_epochs":2,
        "lr":3e-4,
        "lr_decay":False,
        "reg":None,
        "batch_size":2048,
        "num_neg":1,
        "use_tf":True,
        "dropout_rate":0.0,
        "recent_num":10,
        "tf_sess_config":None}},neg_samples=False)


total params: 35,928 | embedding params: 34,136 | network params: 1,792
Training start time: 2023-02-08 12:48:52


train: 100%|██████████| 38/38 [00:06<00:00,  5.54it/s]


Epoch 1 elapsed: 6.876s
	 train_loss: 0.6779


eval_listwise: 100%|██████████| 31/31 [00:00<00:00, 115.80it/s]


	 eval precision@10: 0.0834
	 eval recall@10: 0.0467
	 eval map@10: 0.1874
	 eval ndcg@10: 0.2710


...

Epoch 2 elapsed: 1.778s
	 train_loss: 0.6273


eval_listwise: 100%|██████████| 31/31 [00:00<00:00, 188.71it/s]


	 eval precision@10: 0.1150
	 eval recall@10: 0.0665
	 eval map@10: 0.2158
	 eval ndcg@10: 0.3239


eval_listwise: 100%|██████████| 135/135 [00:00<00:00, 786.08it/s]


Evaluation Results for RNN4REC:
{'precision': 0.11495227995758218, 'recall': 0.0665062245327051, 'map': 0.21576215304078505, 'ndcg': 0.32392041512799913}


#### Recommendations to users

In [ ]:
rnn4recmodel=models['rnn4rec']

In [ ]:
userids=u_users[0]
itemids=u_items[0]

In [ ]:
print('recommendation user '+str(userids))
results= recommend(rnn4recmodel,1, 10)
print(results)

recommendation user 196
{1: array([195, 300, 286, 462, 204, 475, 433, 657, 508, 310])}


In [ ]:
print(" 10 most similar items for item : ", similarItems(rnn4recmodel,itemids, k=10))

 10 most similar items for item :  [289, 568, 530, 185, 208, 141, 429, 294, 47, 202]


##  Caser

In [ ]:
IS_SEQUENTIAL="YES"

In [ ]:

models, evals = run(rating_df, algo=["Caser"], hyperparam={"Caser":{ 
        "loss_type":"cross_entropy",
        "embed_size":16,
        "n_epochs":20,
        "lr":3e-4,
        "lr_decay":False,
        "reg":None,
        "batch_size":512,
        "num_neg":1,
        "use_bn":True,  
        "nh_filters":32,
        "nv_filters":4,
        "dropout_rate":0.0,
        "recent_num":10,
        "tf_sess_config":None}},neg_samples=False)


total params: 100,356 | embedding params: 66,012 | network params: 34,344
Training start time: 2023-02-08 12:51:15


train: 100%|██████████| 151/151 [00:03<00:00, 48.96it/s]


Epoch 1 elapsed: 3.103s
	 train_loss: -0.0605


eval_listwise: 100%|██████████| 31/31 [00:00<00:00, 271.60it/s]


	 eval precision@10: 0.1368
	 eval recall@10: 0.0792
	 eval map@10: 0.3010
	 eval ndcg@10: 0.4048


...

Epoch 20 elapsed: 2.532s
	 train_loss: -4670554.5


eval_listwise: 100%|██████████| 31/31 [00:00<00:00, 221.32it/s]


	 eval precision@10: 0.1963
	 eval recall@10: 0.1219
	 eval map@10: 0.3592
	 eval ndcg@10: 0.4898


eval_listwise: 100%|██████████| 135/135 [00:00<00:00, 821.50it/s]


Evaluation Results for Caser:
{'precision': 0.19628844114528105, 'recall': 0.12187972246931338, 'map': 0.35918519239909935, 'ndcg': 0.48977054318220886}


#### Recommendations to users

In [ ]:
rnn4recmodel=models['Caser']

In [ ]:
userids=u_users[0]
itemids=u_items[0]

In [ ]:
print('recommendation user '+str(userids))
results= recommend(rnn4recmodel,1, 10)
print(results)

recommendation user 196
{1: array([288, 286, 300, 294,   7, 172, 313, 204,  79, 195])}


In [ ]:
print(" 10 most similar items for item : ", similarItems(rnn4recmodel,itemids, k=10))

 10 most similar items for item :  [50, 181, 100, 258, 174, 98, 286, 1, 56, 127]
